In [37]:
import numpy as np
import pandas as pd
import random, copy, os
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from multiprocessing import Pool
import multiprocessing
%run algorithm_functions.ipynb

from cvxopt.base import matrix as m
from cvxopt import solvers
from cvxopt.modeling import op, dot, variable, max, min

## Prepare Data

In [38]:
df = pd.read_csv('trip_data_1_filtered.csv')
df.head()

,medallion,hack_license,pickup_datetime,dropoff_datetime,trip_time_in_secs,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,driver_race,driver_gender,requests_gender,requests_race,pickup_lat_bin,pickup_long_bin,dropoff_lat_bin,dropoff_long_bin
0,89D227B655E5C82AECF13C3F540D4CF4,BA96DE419E711691B9445D6A6307C170,2013-01-01 15:11:48,2013-01-01 15:18:10,382,1.0,-73.978165,40.757977,-73.989838,40.751171,black,m,f,black,7,20,7,19
1,0BD7C8F5BA12B88E0B67BED28BEA73D8,9FD8F69F0804BDB5549F40E9DA1BE472,2013-01-06 00:18:35,2013-01-06 00:22:54,259,1.5,-74.006683,40.731781,-73.994499,40.750660,black,m,f,black,6,19,7,19
2,0BD7C8F5BA12B88E0B67BED28BEA73D8,9FD8F69F0804BDB5549F40E9DA1BE472,2013-01-05 18:49:41,2013-01-05 18:54:23,282,1.1,-74.004707,40.737770,-74.009834,40.726002,black,m,f,black,6,19,6,19
3,DFD2202EE08F7A8DC9A57B02ACB81FE2,51EE87E3205C985EF8431D850C786310,2013-01-07 23:54:15,2013-01-07 23:58:20,244,0.7,-73.974602,40.759945,-73.984734,40.759388,black,m,m,black,7,20,7,19
4,DFD2202EE08F7A8DC9A57B02ACB81FE2,51EE87E3205C985EF8431D850C786310,2013-01-07 23:25:03,2013-01-07 23:34:24,560,2.1,-73.976250,40.748528,-74.002586,40.747868,black,m,m,black,6,20,6,19


In [39]:
df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'])
df['dropoff_datetime'] = pd.to_datetime(df['dropoff_datetime'])

In [40]:
s = MinMaxScaler() # scaling distances such that all values are [0,1]
df['trip_distance'] = s.fit_transform(df['trip_distance'].values.reshape(-1, 1))

In [41]:
min_latitude = min(np.min(df['pickup_latitude']), np.min(df['dropoff_latitude']))
max_latitude = max(np.max(df['pickup_latitude']), np.max(df['dropoff_latitude']))
min_longitude = min(np.min(df['pickup_longitude']), np.min(df['dropoff_longitude']))
max_longitude = max(np.max(df['pickup_longitude']), np.max(df['dropoff_longitude']))

In [42]:
min_longitude, max_longitude, min_latitude, max_latitude

(-74.98333000000002, -73.011703, 40.400002, 40.999813)

In [43]:
requests_features = ['pickup_lat_bin', 'pickup_long_bin', 'dropoff_lat_bin', 'dropoff_long_bin', 
                     'requests_gender', 'requests_race']
# requests_features = ['pickup_lat_bin', 'pickup_long_bin', 'dropoff_lat_bin', 'dropoff_long_bin']
driver_features = ['driver_race', 'driver_gender', 'pickup_lat_bin', 'pickup_long_bin']

###### Total number of drivers and types of drivers

In [44]:
all_driver_types = df.groupby(driver_features)['driver_gender'].agg('count')
print ("Total number of drivers: {}, total driver types: {}".format(len(set(list(df['hack_license']))), 
                                                                    len(all_driver_types)))

Total number of drivers: 32092, total driver types: 750


###### Total number and types of requests

In [45]:
all_request_types = df.groupby(requests_features)['requests_gender'].agg('count')
print ("Total requests: {}, types of requests: {}, total rides: {}".format(all_request_types.sum(), 
                                                                           len(all_request_types),
                                                                           len(df)))

Total requests: 14486242, types of requests: 8676, total rides: 14486242


In [46]:
peak_hour = 16
peak_day = 31

In [47]:
peak_hour_entries = df[np.logical_and(df['pickup_datetime'].dt.hour == peak_hour, 
                                     df['pickup_datetime'].dt.day == peak_day)]
peak_hour_entries.head()

,medallion,hack_license,pickup_datetime,dropoff_datetime,trip_time_in_secs,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,driver_race,driver_gender,requests_gender,requests_race,pickup_lat_bin,pickup_long_bin,dropoff_lat_bin,dropoff_long_bin
7859875,4D3DDD6973BACE91587C7CD79983BC4F,386CB62371C5CED8205E5C51E2E3BBDD,2013-01-31 16:59:00,2013-01-31 17:13:00,840,0.0245,-73.885246,40.773052,-73.913620,40.768368,black,m,f,black,7,21,7,21
7934562,03A23771C873D52AD9FA0B22CEB1B0FB,5B6CCC2D5ECDF7A3D5C3CA74DB82038B,2013-01-31 16:34:54,2013-01-31 16:41:29,394,0.0110,-73.992783,40.742748,-74.004730,40.741692,black,m,m,white,6,19,6,19
7935512,311518AFA6CB5DF4A6544D3BE4B04DBE,884E2B8E2B46991E9314D2119B8DDD20,2013-01-31 16:22:48,2013-01-31 16:32:45,597,0.0110,-73.978783,40.750889,-73.975159,40.759354,black,m,f,white,7,20,7,20
7936838,F687F5186616DA9897728700E265F186,CF2EDF8923B8DD33D100CDEE81CCFB6B,2013-01-31 16:30:14,2013-01-31 16:40:16,601,0.0170,-73.917206,40.761272,-73.919685,40.774532,white,m,m,black,7,21,7,21
7940959,964D6D22CA15493BC4579345085B36DF,583BC216AF18F2358731CD013D7FDDE1,2013-01-31 16:15:13,2013-01-31 16:36:51,1297,0.0590,-74.015930,40.709587,-73.982811,40.741882,black,m,f,black,6,19,6,20


In [48]:
len(peak_hour_entries)

20701

In [49]:
# historical_data = df[df['pickup_datetime'].dt.day != peak_day]
# print (len(historical_data))
# historical_data.head()

## Get Off Peak Hour Driver and Requests

##### Number of drivers in peak hour

In [50]:
peak_hour_driver_types = list(peak_hour_entries.groupby(driver_features)['hack_license'].agg('count').index)
print ("Driver Types: {}, Total Drivers: {}".format(len(peak_hour_driver_types), 
                                                    len(set(list(peak_hour_entries['hack_license'])))))
# assert len(set(list(peak_hour_drivers.values))) == len(peak_hour_drivers)

Driver Types: 83, Total Drivers: 8386


###### Number of request types in the peak hour and number of requests

In [51]:
peak_hour_requests = list(peak_hour_entries.groupby(requests_features)['requests_gender'].agg('count').index)
print ("Request Types: {}, Total requests: {}".format(len(peak_hour_requests), 
                                                      len(peak_hour_entries)))

Request Types: 686, Total requests: 20701


In [52]:
def calculate_probability(request, driver, kappa=0.5):
    if request.race == 'black' and driver.race == 'black':
        request_driver_coeff = 0.3
    elif request.race == 'white' and driver.race == 'white':
        request_driver_coeff = 0.6
#     elif request.race == 'black' and driver.race == 'white':
#         request_driver_coeff = 0.1
    else:
        request_driver_coeff = 0.1
#     p = 0.6 * request.distance + 0.4 * request_driver_coeff
    p = request_driver_coeff
    p = kappa + (1 - kappa) * p
    assert p <= 1 and p >= kappa
    return p

###### Subsample drivers for off-peak hour: assign k (=2) drivers for each driver type

In [53]:
def get_drivers(driver_Bu):
    subsample_fraction = 1
    k = 80
    random.seed(42)
    sampled_peak_hour_drivers = random.sample(peak_hour_driver_types, k)

    # Further downsampling to 4 driver types!
#     sampled_peak_hour_drivers = random.sample(sampled_peak_hour_drivers, k=20)

    print ("Chosen driver types: {}".format(len(sampled_peak_hour_drivers)))

    drivers = [] # Equivalent to set U
    for driver_type in sampled_peak_hour_drivers:
        mask = None
        bool_peak_entries_match = peak_hour_entries[driver_features] == driver_type
        for ft in driver_features:
            mask = bool_peak_entries_match[ft].values if mask is None else\
                np.logical_and(mask, bool_peak_entries_match[ft].values)

        if (np.count_nonzero(mask)) < 2:
            continue
        sampled_ids = random.sample(list(peak_hour_entries[mask]['hack_license']), k=k \
                                    if k is not None and k < np.count_nonzero(mask) else np.count_nonzero(mask))
        for d_id in sampled_ids:
            driver_instance = Driver(**dict(zip(driver_features, driver_type)), d_id=d_id, Bu=random.randint(1,driver_Bu))
            if driver_instance not in drivers:
#                 driver_instance.race = np.random.choice(['black', 'white'], p=[0.75, 0.25])[0]
#                 print (driver_instance.race)
                drivers.append(driver_instance)
            break
            
    set_unique_ids(drivers)

    print ("Total number of drivers: {}, Size of set U: {}".format(len(drivers), len(set(drivers))))
    return drivers

###### Subsample requests for the peak hour: multiply arrival rate of requests by k from each request type

In [54]:
def count_total_requests(all_entries, sampled_request_types):
    total = 0
    for request_vals in sampled_request_types:
        mask = None
        bool_peak_entries_match = all_entries[requests_features] == request_vals
        for ft in requests_features:
            mask = np.array(bool_peak_entries_match[ft]) if mask is None else \
                mask & np.array(bool_peak_entries_match[ft])
        total += np.count_nonzero(mask)
    return total

In [55]:
def get_requests(request_quota):
    subsample_fraction = 0.2
#     k = 1
    random.seed(42)
    sampled_request_types = random.sample(peak_hour_requests, k=int(subsample_fraction * len(peak_hour_requests)))
    print ("Chosen types of requests: {}".format(len(sampled_request_types)))
    total_sampled_requests = count_total_requests(peak_hour_entries, sampled_request_types)
    if subsample_fraction == 1:
        assert total_sampled_requests == len(peak_hour_entries)
    print ("Total Sampled Requests: {}".format(total_sampled_requests))

    requests = [] # Equivalent to set V
    for request_vals in sampled_request_types:
        mask = None
        bool_peak_entries_match = peak_hour_entries[requests_features] == request_vals
        for ft in requests_features:
            mask = np.array(bool_peak_entries_match[ft]) if mask is None else \
                mask & np.array(bool_peak_entries_match[ft])
        # Find average distance in miles and put that in request type
        requests.append(Request(**dict(zip(requests_features, request_vals)), 
                                arrival_rate=float(np.count_nonzero(mask)),
                                distance=np.mean(peak_hour_entries[mask]['trip_distance']),
                                quota=request_quota))

    set_unique_ids(requests)

    print ("Initial Number of Requests: {}, size of set V: {}".format(
        np.sum([r.arrival_rate for r in requests]), len(set(requests))))
    return requests, np.sum([r.arrival_rate for r in requests])

In [56]:
def edge_existence_bool(driver, request):
    return abs(driver.latitude-request.start_latitude)+abs(driver.longitude-request.start_longitude)

def draw_probability_edges(drivers, requests):
    edge_matrix = np.zeros((len(drivers), len(requests))) - 1
    for d in drivers:
        for r in requests:
            if edge_existence_bool(d, r) < 2:
                edge_matrix[d.u_id, r.u_id] = calculate_probability(r, d)
    return edge_matrix

In [57]:
def filter_out_drivers_requests(drivers, requests, total_requests):
    probability_matrix = draw_probability_edges(drivers, requests)
    new_requests, new_total = [], total_requests
    average_arrival_rate = total_requests/len(requests)
#     for r in requests:
#         if np.count_nonzero(probability_matrix[:,r.u_id] != -1) > 20 and r.arrival_rate >= average_arrival_rate:
#             if abs(r.arrival_rate - average_arrival_rate) > 10:
#                 new_arrival_rate = int(average_arrival_rate) + random.randint(0, 10)
#                 new_total -= (r.arrival_rate - new_arrival_rate)
#                 r.arrival_rate = new_arrival_rate
#             new_requests.append(r)
#         else:
#             new_total -= r.arrival_rate
#     for i in range(len(drivers)):
#         if np.count_nonzero(probability_matrix[drivers[i].u_id,:] != -1) > 0:
#             new_drivers.append(drivers[i])

    for i in range(len(requests)):
        if np.count_nonzero(probability_matrix[:,requests[i].u_id] != -1) > 3:
            new_requests.append(requests[i])
        else:
            new_total -= requests[i].arrival_rate
    np.random.seed(42)
    new_arrival_rates = np.random.normal(loc=5, scale=1, size=len(new_requests))
    for i in range(len(new_requests)):
        new_total -= new_requests[i].arrival_rate
        new_requests[i].arrival_rate = round(new_arrival_rates[i])
        new_total += new_requests[i].arrival_rate
        if new_requests[i].race == 'white':
            new_requests[i].arrival_rate += new_requests[i].arrival_rate
            new_total += new_requests[i].arrival_rate
    assert int(new_total) == new_total
    
    set_unique_ids(new_requests)
    
    probability_matrix = draw_probability_edges(drivers, new_requests)
    new_drivers = []
    for i in range(len(drivers)):
        if np.count_nonzero(probability_matrix[drivers[i].u_id,:] != -1) > 0:
            new_drivers.append(drivers[i])
    
    return new_requests, int(new_total), new_drivers

In [58]:
def get_drivers_and_requests(driver_Bu, request_quota):
    drivers = get_drivers(driver_Bu)
    requests, total_sampled_requests = get_requests(request_quota)
    requests, total_sampled_requests, drivers = filter_out_drivers_requests(drivers, requests, total_sampled_requests)
    set_unique_ids(drivers)
    for i in range(len(drivers)):
        if drivers[i].race == 'white':
            drivers[i].Bu = 2 * drivers[i].Bu
#     set_unique_ids(requests)
    print ("Final Number of Requests: {} {}, size of set V: {}".format(
        np.sum([r.arrival_rate for r in requests]), total_sampled_requests, len(set(requests))))
    print ("Final Number of Drivers: {}, size of set U: {}".format(
        np.sum([r.Bu for r in drivers]), len(set(drivers))))
    return drivers, requests

In [59]:
# def get_drivers_requests(driver_Bu,request_quota):
#     random.seed(42)
#     sampled_request_types = random.sample(peak_hour_requests, k=20)
#     print ("Chosen types of requests: {}".format(len(sampled_request_types)))

#     drivers = []
#     requests = [] # Equivalent to set V
# #     filter_features = ['pickup_lat_bin', 'pickup_long_bin', 'dropoff_lat_bin', 'dropoff_long_bin']
#     d_id_index = 1
#     for request_vals in sampled_request_types:
#         mask = None
# #         print(request_vals[0])
#         bool_peak_entries_match = peak_hour_entries[requests_features] == request_vals
#         for ft in requests_features:
#             mask = np.array(bool_peak_entries_match[ft]) if mask is None else \
#                 mask & np.array(bool_peak_entries_match[ft])
# #         new_arrival_rate = np.random.normal(loc=5, scale=1)
#         new_arrival_rate = random.randint(4,6)
#         # Find average distance in miles and put that in request type
#         requests.append(Request(pickup_lat_bin=request_vals[0], 
#                                 pickup_long_bin=request_vals[1], 
#                                 dropoff_lat_bin=request_vals[2], 
#                                 dropoff_long_bin=request_vals[3],
#                                 requests_race='black',
#                                 requests_gender='m',
#                                 arrival_rate=new_arrival_rate,
#                                 distance=np.mean(peak_hour_entries[mask]['trip_distance']),
#                                 quota=random.randint(1,request_quota)))
#         requests.append(Request(pickup_lat_bin=request_vals[0], 
#                                 pickup_long_bin=request_vals[1], 
#                                 dropoff_lat_bin=request_vals[2], 
#                                 dropoff_long_bin=request_vals[3],
#                                 requests_race='black',
#                                 requests_gender='f',
#                                 arrival_rate=new_arrival_rate,
#                                 distance=np.mean(peak_hour_entries[mask]['trip_distance']),
#                                 quota=random.randint(1,request_quota)))
#         requests.append(Request(pickup_lat_bin=request_vals[0], 
#                                 pickup_long_bin=request_vals[1], 
#                                 dropoff_lat_bin=request_vals[2], 
#                                 dropoff_long_bin=request_vals[3],
#                                 requests_race='white',
#                                 requests_gender='m',
#                                 arrival_rate=2*new_arrival_rate,
#                                 distance=np.mean(peak_hour_entries[mask]['trip_distance']),
#                                 quota=random.randint(1,request_quota)))
#         requests.append(Request(pickup_lat_bin=request_vals[0], 
#                                 pickup_long_bin=request_vals[1], 
#                                 dropoff_lat_bin=request_vals[2], 
#                                 dropoff_long_bin=request_vals[3],
#                                 requests_race='white',
#                                 requests_gender='f',
#                                 arrival_rate=2*new_arrival_rate,
#                                 distance=np.mean(peak_hour_entries[mask]['trip_distance']),
#                                 quota=random.randint(1,request_quota)))
#         B=random.randint(1,driver_Bu)
#         driver_instance = Driver(pickup_lat_bin=request_vals[0],pickup_long_bin=request_vals[1],driver_race='white',driver_gender='m',d_id=d_id_index,Bu=B)
#         if driver_instance not in drivers:
#             drivers.append(driver_instance)
#             d_id_index = d_id_index + 1
#             drivers.append(Driver(pickup_lat_bin=request_vals[0],pickup_long_bin=request_vals[1],driver_race='white',driver_gender='f',d_id=d_id_index,Bu=B))
#             d_id_index = d_id_index + 1
#             drivers.append(Driver(pickup_lat_bin=request_vals[0],pickup_long_bin=request_vals[1],driver_race='black',driver_gender='m',d_id=d_id_index,Bu=3*B))
#             d_id_index = d_id_index + 1
#             drivers.append(Driver(pickup_lat_bin=request_vals[0],pickup_long_bin=request_vals[1],driver_race='black',driver_gender='f',d_id=d_id_index,Bu=3*B))
#         changelist = [[1,0],[0,1],[-1,0],[0,-1]]
#         tmp_vals = random.choice(changelist)
#         another_driver_lat = request_vals[0] + tmp_vals[0]
#         another_driver_long = request_vals[1] + tmp_vals[1]
# #         if another_driver_lat < 1 || another_driver_lat > :
# #             another_driver_lat = another_driver_lat + 1
#         B=random.randint(1,driver_Bu)
#         driver_instance = Driver(pickup_lat_bin=another_driver_lat,pickup_long_bin=another_driver_long,driver_race='white',driver_gender='m',d_id=d_id_index,Bu=B)
#         if driver_instance not in drivers:
#             drivers.append(driver_instance)
#             d_id_index = d_id_index + 1
#             drivers.append(Driver(pickup_lat_bin=another_driver_lat,pickup_long_bin=another_driver_long,driver_race='white',driver_gender='f',d_id=d_id_index,Bu=B))
#             d_id_index = d_id_index + 1
#             drivers.append(Driver(pickup_lat_bin=another_driver_lat,pickup_long_bin=another_driver_long,driver_race='black',driver_gender='m',d_id=d_id_index,Bu=3*B))
#             d_id_index = d_id_index + 1
#             drivers.append(Driver(pickup_lat_bin=another_driver_lat,pickup_long_bin=another_driver_long,driver_race='black',driver_gender='f',d_id=d_id_index,Bu=3*B))
            
#     set_unique_ids(requests)
#     set_unique_ids(drivers)

#     print ("Total Requests: {}, size of set V: {}, Total Drivers:{}, size of set U:{}".format(
#         np.sum([r.arrival_rate for r in requests]), len(set(requests)), np.sum([d.Bu for d in drivers]), len(drivers)))
#     return drivers, requests

In [60]:
# drivers, requests = get_drivers_and_requests(5, 2)
# probability_matrix = draw_probability_edges(drivers, requests)

In [61]:
# for d in drivers:
#  print(d)
# print(len(drivers))
# print(requests[8])
# print(probability_matrix[:,8])

In [62]:
# for r in requests:
#     print(r)
# print(len(requests))

In [63]:
# drivers, requests = get_drivers_and_requests(15, 3)
# probability_matrix = draw_probability_edges(drivers, requests)
# # print(len(drivers))
# # print(len(requests))
# r_vs = []
# for r in requests:
#     r_vs.append(r.arrival_rate)
# min(r_vs), max(r_vs)

# in_degree_v = []
# for i in range(len(requests)):
#     in_degree_v.append(np.count_nonzero(probability_matrix[:,i] != -1))

# in_degree_u = []
# for i in range(len(drivers)):
#     in_degree_u.append(np.count_nonzero(probability_matrix[i,:] != -1))
# #     print(np.count_nonzero(probability_matrix[i,:] != -1))

# print ("Drivers' in-degree, average: {}, min: {}, max: {}\nRiders' in-degree, average: {}, min: {}, max: {}"\
#        .format(np.mean(in_degree_u), min(in_degree_u), max(in_degree_u), 
#                np.mean(in_degree_v), min(in_degree_v), max(in_degree_v)))